In [24]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import os
import matplotlib.pyplot as plt    
import seaborn as sns      

In [2]:
to_train = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_dataset.csv')
data = to_train.copy()
priority = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_priority.csv')

In [3]:
data

Issue_id Priority                                              Title  \
0             1       P3    Usability issue with external editors (1GE6IRL)   
1            13       P3          CC Discussion: local versioning (1GAT3PL)   
2            12       P3     Manage/unmanage support and policies (1GALAEG)   
3            11       P3  API: ISharingManager::load mapping vcm project...   
4            10       P3             API - VCM event notification (1G8G6RR)   
...         ...      ...                                                ...   
68119    229777       P3  [Viewers] Wrong argument in the first statemen...   
68120    229779       P3                           NPE in performance tests   
68121    229782       P3                 Performance tests for ICU Collator   
68122    229789       P3  [Examples] examples plugins create duplicate m...   
68123    229792       P3  Collapse All and Expand All buttons are separated   

                                             Description Component  \
0      - Setup a project that contains a *.gif resour...      Team   
1      Would it make sense to have a (the?) projects ...      Team   
2      Currently every new resource added to a shared...      Team   
3      Jean-Michel (08/02/2001 1:38:48 PM); ; The ISh...      Team   
4      There seems to be a need for a VCM Event notif...      Team   
...                                                  ...       ...   
68119  Build ID: 3.3 (I am not sure what You mean by ...        UI   
68120  There are several NPEs within the UI session t...        UI   
68121  I20080501-0100; ; Now that were using the coll...        UI   
68122  Created attachment 98318; screenshot; ; I20080...        UI   
68123  Build ID: M20080221-1800; ; Steps To Reproduce...      Team   

                                           Combined_Text  
0      ['Usability', 'issue', 'external', 'editors', ...  
1      ['CC', 'Discussion', 'local', 'versioning', '1...  
2      ['Manage/unmanage', 'support', 'policies', '1G...  
3      ['API', 'ISharingManager', ':load', 'mapping',...  
4      ['API', 'VCM', 'event', 'notification', '1G8G6...  
...                                                  ...  
68119  ['Viewers', 'Wrong', 'argument', 'first', 'sta...  
68120  ['NPE', 'performance', 'tests', 'several', 'NP...  
68121  ['Performance', 'tests', 'ICU', 'Collator', 'I...  
68122  ['Examples', 'examples', 'plugins', 'create', ...  
68123  ['Collapse', 'Expand', 'buttons', 'separated',...  

[68124 rows x 6 columns]

In [4]:
data['Combined_Text'][0:1]

0    ['Usability', 'issue', 'external', 'editors', ...
Name: Combined_Text, dtype: object

In [5]:
texts = data['Combined_Text'].tolist()

In [6]:
vectorizer = CountVectorizer() #Part of Sklearn, converts into matrix of token counts

In [7]:
x = vectorizer.fit_transform(texts) #turns data into vectors

In [8]:
lda = LatentDirichletAllocation(n_components=10,
    random_state=0) #Specifics of this model, ten topics

In [9]:
lda.fit(x) #Run through our data

LatentDirichletAllocation(random_state=0)

In [10]:
topic_distributions = lda.transform(x) #Method to get topic dists for each bug
topics = []

In [11]:
for topic_dist in topic_distributions:
    dominant_topic = topic_dist.argmax()
    topics.append(dominant_topic)

In [12]:
data['Topic'] = topics

In [13]:
data

Issue_id Priority                                              Title  \
0             1       P3    Usability issue with external editors (1GE6IRL)   
1            13       P3          CC Discussion: local versioning (1GAT3PL)   
2            12       P3     Manage/unmanage support and policies (1GALAEG)   
3            11       P3  API: ISharingManager::load mapping vcm project...   
4            10       P3             API - VCM event notification (1G8G6RR)   
...         ...      ...                                                ...   
68119    229777       P3  [Viewers] Wrong argument in the first statemen...   
68120    229779       P3                           NPE in performance tests   
68121    229782       P3                 Performance tests for ICU Collator   
68122    229789       P3  [Examples] examples plugins create duplicate m...   
68123    229792       P3  Collapse All and Expand All buttons are separated   

                                             Description Component  \
0      - Setup a project that contains a *.gif resour...      Team   
1      Would it make sense to have a (the?) projects ...      Team   
2      Currently every new resource added to a shared...      Team   
3      Jean-Michel (08/02/2001 1:38:48 PM); ; The ISh...      Team   
4      There seems to be a need for a VCM Event notif...      Team   
...                                                  ...       ...   
68119  Build ID: 3.3 (I am not sure what You mean by ...        UI   
68120  There are several NPEs within the UI session t...        UI   
68121  I20080501-0100; ; Now that were using the coll...        UI   
68122  Created attachment 98318; screenshot; ; I20080...        UI   
68123  Build ID: M20080221-1800; ; Steps To Reproduce...      Team   

                                           Combined_Text  Topic  
0      ['Usability', 'issue', 'external', 'editors', ...      5  
1      ['CC', 'Discussion', 'local', 'versioning', '1...      5  
2      ['Manage/unmanage', 'support', 'policies', '1G...      5  
3      ['API', 'ISharingManager', ':load', 'mapping',...      5  
4      ['API', 'VCM', 'event', 'notification', '1G8G6...      5  
...                                                  ...    ...  
68119  ['Viewers', 'Wrong', 'argument', 'first', 'sta...      7  
68120  ['NPE', 'performance', 'tests', 'several', 'NP...      7  
68121  ['Performance', 'tests', 'ICU', 'Collator', 'I...      7  
68122  ['Examples', 'examples', 'plugins', 'create', ...      9  
68123  ['Collapse', 'Expand', 'buttons', 'separated',...      9  

[68124 rows x 7 columns]

In [14]:
data.groupby('Topic').count()

Issue_id  Priority  Title  Description  Component  Combined_Text
Topic                                                                  
0          2832      2832   2832         2832       2832           2832
1          2349      2349   2349         2349       2349           2349
2           348       348    348          348        348            348
3           450       450    450          450        450            450
4          4817      4817   4817         4817       4817           4817
5          8306      8306   8306         8306       8306           8306
6           782       782    782          782        782            782
7         15895     15895  15895        15895      15895          15895
8          4865      4865   4865         4865       4865           4865
9         27480     27480  27480        27480      27480          27480

In [34]:

extract_dir = '/home/rpierson/PiersonREU/extracted'
data_file = os.path.join(extract_dir, 'train_dataset.csv')
data.to_csv(data_file, index = False)

In [20]:
gr = data.groupby('Topic')['Combined_Text'].count()
gr

Topic
0     2832
1     2349
2      348
3      450
4     4817
5     8306
6      782
7    15895
8     4865
9    27480
Name: Combined_Text, dtype: int64

In [33]:
plt.xticks(rotation = 0)
plt.grid(axis='y', color='black', alpha=0.3)
plt.title('Total Number of Training Data Bug Reports in Each Topic')

# Here we use plot the grouped index on the x axis and the values on the y axis
sns.barplot(x = gr.index, y = gr.values)

plt.show()